In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment
from defdap.quat import Quat

from pathlib import Path

import copy 
import pandas as pd
import datetime

from scipy.signal import find_peaks
from scipy.linalg import sqrtm, polar


import os

# get dictools stuff 
import sys
sys.path.append("c:/work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

c:\work\defdap\defdap\hrdic.py:525: SyntaxWarning: invalid escape sequence '\/'
  print('Filtering will remove {0} \/ {1} ({2:.3f} %) datapoints in map'
c:\work\defdap\defdap\hrdic.py:528: SyntaxWarning: invalid escape sequence '\/'
  'Filtering will remove {0} \/ {1} ({2:.3f} %) datapoints in cropped map'
Loading ITKPyBase... Loading ITKPyBase... Loading ITKCommon... Loading ITKCommon... Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageFrequency... Loading ITKImageFrequency... Loading ITKIOImageBase... Loading ITKIOBMP... Loading ITKIOBMP... Loading ITKIOBioRad... Loading ITKIOBioRad... Loading ITKIOBruker... Loading ITKIOBruker... Loading ITKIOGDCM... Loading ITKIOGDCM... Loading ITKIOIPL... Loading ITKIOIPL... Loading ITKIOGE... Loading ITKIOGE... Loading ITKIOGIPL... Loading ITKIOGIPL... Loading ITKIOHDF5... Loading ITKIOHDF5... Loading ITKIOJPEG... Loading I

<itkTemplate itk::TileConfiguration>
Options:
  [2,]
  [3,]
  [4,]


In [2]:
exp = experiment.Experiment()

# load DIC data 
data_dir = Path('.')
dic_frame = experiment.Frame()
for dic_file in sorted(data_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)

Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)


In [3]:
ebsd_frame = experiment.Frame()
ebsd.Map(data_dir / 'Post_EBSD/map_small.cpr',
         increment=exp.increments[0], frame=ebsd_frame)

Loaded EBSD data (dimensions: 1825 x 1750 pixels, step size: 0.2 um)


In [4]:
ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.set_homog_point()

dic_map = exp.increments[0].maps['hrdic']
# dic_map.set_homog_point(vmin=0,vmax=0.1)

In [5]:
ebsd_frame.homog_points = [(274, 381),
 (891, 265),
 (1613, 265),
 (958, 760),
 (1148, 1241),
 (1425, 1035),
 (378, 1109),
 (390, 1409),
 (1615, 1407),
 (889, 1245),
 (710, 635),
 (576, 558),
 (556, 696),
 (318, 703),
 (545, 499),
 (1216, 958),
 (1375, 752),
 (781, 976),
 (1303, 732),
 (1061, 746),
 (1514, 594),
 (1274, 510),
 (1435, 427),
 (1117, 664),
 (425, 1075),
 (631, 1400),
 (631, 1036),
 (769, 1126),
 (1152, 942),
 (1240, 1339),
 (1528, 1149),
 (1084, 1193)]


dic_frame.homog_points = [(83, 363),
 (856, 170),
 (1759, 146),
 (949, 829),
 (1186, 1442),
 (1536, 1173),
 (233, 1304),
 (245, 1687),
 (1761, 1668),
 (868, 1444),
 (634, 672),
 (465, 576),
 (444, 757),
 (144, 785),
 (425, 495),
 (1273, 1087),
 (1470, 805),
 (727, 1117),
 (1382, 780),
 (1074, 814),
 (1643, 593),
 (1341, 480),
 (1543, 362),
 (1147, 700),
 (290, 1256),
 (548, 1663),
 (545, 1195),
 (719, 1310),
 (1194, 1069),
 (1303, 1594),
 (1654, 1333),
 (1109, 1382)]

In [6]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.link_ebsd_map(ebsd_map, transform_type="projective")
    



    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs='line'
    )
    plt.tight_layout()

Finished building quaternion array (0:00:16) 
Finished finding grain boundaries (0:00:49) 


In [7]:
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.plot_map(
        'r_ang'
    )

In [9]:
# find special boundaries 
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)


# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# calculate neighbour network
ebsd_map.build_neighbour_network()

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
twin_lines = []
for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False

    # calculate grain ref orientation
    grain1.calc_average_ori()
    grain2.calc_average_ori()

    misori = grain2.ref_ori * grain1.ref_ori.conjugate
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines.append(b_seg)

# make into boundary set
s_bounds = ebsd.BoundarySet.from_boundary_segments(twin_lines)

Finished finding grains (0:00:07) twork..
Finished constructing neighbour network (0:00:09) 


In [ ]:
plot = ebsd_map.plot_ipf_map([1,0,0],plot_gbs='line',dilate_boundaries=True)
plot.add_grain_boundaries(kind='line',boundaries=s_bounds,colour='red',linewidths=10)


In [ ]:
s_bounds_dic = hrdic.BoundarySet.from_ebsd_boundaries(dic_map,s_bounds)

In [ ]:
plot = dic_map.plot_map('max_shear',vmin=0,vmax=0.1,plot_gbs='pixel',dilate_boundaries=True)
plot.add_grain_boundaries(kind='pixel',boundaries=s_bounds_dic,dilate=True,color='limegreen',alpha=1)

In [ ]:
dic_map.data.max_shear < np.percentile(dic_map.data.max_shear,95)

In [ ]:
mask = dic_map.generate_threshold_mask(dic_map.data.max_shear < np.percentile(dic_map.data.max_shear,98))
dic_map.apply_threshold_mask()

In [ ]:
gb_im = skimage.morphology.dilation(s_bounds_dic.image,footprint=np.ones((3,3)))

gb_im_to_plot = gb_im.astype(float)

gb_im_to_plot[~gb_im] = np.nan

plot = dic_map.plot_map()#,plot_gbs='pixel',dilate_boundaries=True)
plot.add_map(gb_im_to_plot,alpha=0.5,cmap='Greens')

In [ ]:
# test with a single map to make things easier
# exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
dic_map = hrdic.Map(dic_dir/ 'map_44.txt', data_type='openpiv')

hfw = 20.0 # microns
pixelwidth = 2048
pixelsize = hfw/pixelwidth


# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)

dic_map.set_scale(pixelsize)
dic_map.set_crop(left=100,right=100,top=100,bottom=100)

ebsd_pre_frame = experiment.Frame()
ebsd_map=ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

ebsd_map.find_grains()
ebsd_map.build_neighbour_network()

In [ ]:
# exp = experiment.Experiment()

# dic_dir = Path('./DIC/')
# ebsd_pre_dir = Path('./Pre_EBSD/')
# ebsd_post_dir = Path('./Post_EBSD/')

# dic_frame = experiment.Frame()
# for dic_file in sorted(dic_dir.glob('map_*.txt')):
#     hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
# hfw = 20.0 # microns
# pixelwidth = 2048
# pixelsize = hfw/pixelwidth

# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_scale(pixelsize)
#     dic_map.set_crop(left=100,right=100,top=100,bottom=100)
#     # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
#     print(dic_map)
    
# ebsd_pre_frame = experiment.Frame()
# ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.find_boundaries()
# ebsd_map.find_grains
# ebsd_map.build_neighbour_network()

In [ ]:
# ebsd_post_frame = experiment.Frame()
# ebsd.Map(ebsd_post_dir / 'map_small.cpr',increment=exp.increments[-1], frame=ebsd_post_frame)

In [ ]:
exp.increments[1].maps['ebsd']

In [ ]:
# dic_map = exp.increments[0].maps['hrdic']
dic_map.set_homog_point(vmin=0,vmax=0.2)

# ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.set_homog_point()

# dic_map.homog_points = [(1722, 1177),
#  (1335, 1313),
#  (948, 1745),
#  (549, 1661),
#  (245, 1680),
#  (233, 1300),
#  (579, 1158),
#  (443, 756),
#  (559, 136),
#  (83, 359),
#  (1438, 495),
#  (857, 165)]

# ebsd_map.homog_points = [(2541, 1823),
#  (2240, 1932),
#  (1940, 2266),
#  (1631, 2207),
#  (1394, 2225),
#  (1388, 1933),
#  (1659, 1820),
#  (1554, 1519),
#  (1648, 1054),
#  (1279, 1219),
#  (2323, 1319),
#  (1875, 1080)]

# dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
# dic_map.plot_map('max_shear', vmin=0,vmax=0.1,plot_scale_bar=True,plot_gbs=True)
#  # link maps
# for inc, dic_map in exp.iter_over_maps('hrdic'):
#     dic_map.set_crop
#     dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
#     # dic_map.plot_map(
#     #     'max_shear', vmin=0, vmax=0.10, 
#     #     plot_scale_bar=False, plot_gbs='line'
#     # )
    # plt.tight_layout()

In [ ]:
# find twin boundaries
misori_twin = Quat.from_axis_angle([1, 1, 1], 60*np.pi/180)
misori_twin_tol = 5*np.pi/180

# create all symmetric equivalent misorientations
misori_twin_all = []
syms = ebsd_map.primary_phase.crystal_structure.symmetries
for sym_i in syms:
    for sym_j in syms:
        misori_twin_all.append(sym_i.conjugate * misori_twin * sym_j)
# get rid of any duplicates
misori_twin_all = list(set(misori_twin_all))

# loop over all grain boundary segments and check if the misorientation between
# the two grains is within tolerance of the twin misorientation
twin_lines = []
for grain1, grain2, b_seg in ebsd_map.neighbour_network.edges.data('boundary'):
    twin = False
    misori = grain2.ref_ori * grain1.ref_ori.conjugate
    for misori_twin in misori_twin_all:
        if 2 * np.arccos(misori_twin.dot(misori)) < misori_twin_tol:
            twin = True
            break
    
    if not twin:
        continue
    
    twin_lines += b_seg.boundary_lines

In [ ]:
ebsd_map.build_neighbour_network()

In [ ]:
ebsd_map.

In [ ]:
t = ebsd_map.neighbour_network.edges.data('boundary')

In [ ]:
plot = ebsd_map.plot_ipf_map([1,0,0])
plot.add_grain_boundaries(kind='line',boundaries=twin_lines, colour='red')

In [ ]:
 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    # plt.tight_layout()

dic_map = exp.increments[-1].maps['hrdic'] 
dic_map.plot_map(
    
)